# Diabetes - Machine Learning Prediction
Develop and deploy a machine learning model to predict whether an individual is diabetic based on various health metrics. Demonstrate the full lifecycle of a machine learning model, from data preprocessing and model training to deployment and real-time prediction.

## Project Overview
This project develops a machine learning system to predict diabetes status based on health metrics. :

**Goals:**
- **Accurate Prediction:** Create a model that accurately predicts whether an individual is diabetic or non-diabetic based on provided health metrics.
- **Model Deployment:** Develop a system that allows for easy saving, loading, and reuse of the trained model and preprocessing scaler.


## Dataset
This [dataset](https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset) is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

Information about dataset attributes:
- **Pregnancies:** To express the Number of pregnancies
- **Glucose:** To express the Glucose level in blood
- **BloodPressure:** To express the Blood pressure measurement
- **SkinThickness:** To express the thickness of the skin
- **Insulin:** To express the Insulin level in blood
- **BMI:** To express the Body mass index
- **DiabetesPedigreeFunction:** To express the Diabetes percentage
- **Age:** To express the age
- **Outcome:** To express the final result 1 is Yes and 0 is No

## Machine Learning Prediction

**Install dependencies**

In [141]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

**Data Collection & processing**

In [142]:
# Load the dataset

df=pd.read_csv('/kaggle/input/diabetes/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [143]:
df.shape

(768, 9)

In [144]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [146]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [147]:
# 0 = Non-diabetic people
# 1 = Diabetic people

df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [148]:
# Separate data and labels

X=df.drop(columns='Outcome', axis=1)
Y=df['Outcome']

In [149]:
print(X)
print(Y)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [150]:
# Extract column names to be used later when making web app interface
for column in X.columns:
    print(column)

Pregnancies
Glucose
BloodPressure
SkinThickness
Insulin
BMI
DiabetesPedigreeFunction
Age


**Data standardization**

In [151]:
# Instantiate StandardScaler and fit the model

scaler=StandardScaler()

scaler.fit(X)

StandardScaler()

In [152]:
# Transform all data to a common range

standardized_df=scaler.transform(X)

print(standardized_df)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [153]:
# re-assign the standardized data to X

X = standardized_df
Y = df['Outcome']

print(X)
print(Y)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


**Training and Testing the data**

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=2)

In [155]:
print(X.shape,X_train.shape,X_test.shape)

(768, 8) (614, 8) (154, 8)


**Model Training: SupportVectorMachine(SVM)**

In [156]:
classifier = svm.SVC(kernel='linear')

In [157]:
# Training the SVM

classifier.fit(X_train, y_train)

SVC(kernel='linear')

**Model evaluation**

In [158]:
# Find the accuracy score on the  training data
X_train_pred = classifier.predict(X_train)
X_train_accuracy = accuracy_score(X_train_pred, y_train)

In [159]:
print('Accuracy score of the training data: ', X_train_accuracy)

Accuracy score of the training data:  0.7866449511400652


In [160]:
# Find the accuracy score on the  testing data
X_test_pred = classifier.predict(X_test)
X_test_accuracy = accuracy_score(X_test_pred, y_test)

In [161]:
print('Accuracy score of the training data: ', X_test_accuracy)

Accuracy score of the training data:  0.7727272727272727


**Making predictions**

In [162]:
data = (3,126,88,41,235,39.3,0.704,27)

# Change input data to np array
data_as_np_array = np.asarray(data)

# Reshape the array as we are predicting for one instance
data_reshaped = data_as_np_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0]==1):
    print('The person is not diabetic')
else:
    print('The person is diabetic')

[[-0.25095213  0.1597866   0.97680475  1.28363829  1.34758997  0.92745247
   0.70104112 -0.53102292]]
[0]
The person is diabetic


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


**Saving the trained model**

In [163]:
import pickle

In [164]:
# Save the model to a file
filename = 'diabetes_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [165]:
# Save the scaler to a file
scaler_filename = 'diabetes_scaler.sav'
pickle.dump(scaler, open(scaler_filename, 'wb'))

In [166]:
# Loading the saved model

loaded_model = pickle.load(open('diabetes_model.sav', 'rb'))

In [167]:
# Loading the saved scaler
loaded_scaler = pickle.load(open('diabetes_scaler.sav', 'rb'))

In [168]:
data = (3,126,88,41,235,39.3,0.704,27)

# Change input data to np array
data_as_np_array = np.asarray(data)

# Reshape the array as we are predicting for one instance
data_reshaped = data_as_np_array.reshape(1,-1)

# Standardize the input data using the loaded scaler
data_standardized = loaded_scaler.transform(data_reshaped)

prediction = loaded_model.predict(data_standardized)
print(prediction)

if (prediction[0]==1):
    print('The person is not diabetic')
else:
    print('The person is diabetic')

[0]
The person is diabetic


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
